### 

# Rapport : Estimation du Tarif des Concurrents :

Dans le domaine de l’assurance, estimer avec précision les primes commerciales est un défi majeur en raison de la diversité et de la complexité des variables influençant leur calcul. Ces variables incluent des facteurs socio-démographiques (âge, sexe, statut matrimonial), des caractéristiques techniques des véhicules (puissance, type de carburant, âge), et des comportements des assurés (bonus/malus, usage du véhicule). La difficulté réside dans la modélisation efficace de ces variables hétérogènes et leur interaction pour prédire un montant juste, compétitif et adapté aux risques tout en s’adaptant aux évolutions du marché et des comportements des clients.

Le but construire un modèle prédictif pour estimer la prime commerciale des concurrents à partir des données fournies.

# Importation des Bibliothèques 


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Any, Tuple
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import learning_curve
from common_function import(
          load_config ,
          load_data,
          detect_missing_values ,
          save_model_with_directory,
          plot_box_plot,
          plot_feature_distribution,
          plot_correlation_matrix,
          plot_scatter,
          detect_outliers_summary,
          cap_outliers,
          log_transform,
          scale_features,
          preprocess_modeling,
          evaluation_metrics,
          evaluate_on_validation_set,
          
)
import joblib
import os

## Étape 1 : Préparation des données

Charger et comprendre les données brutes afin de déterminer la qualité des données et les caractéristiques de chaque variable

### 1.Chargement de la Configuration et des Données

In [ ]:
""" Charger le fichier de configuration pour accéder aux chemins d'accès et aux paramètres, 
puis  l'utiliser pour charger les données d'entraînement
"""
config = load_config("config\config.yaml") 
data=load_data(config['data']['train_path'])
test_data=load_data(config['data']['test_path'])

In [ ]:
data.info()

Les données comportent des **variables numériques** (***AgeConducteur***, ***BonusMalus***, ***AgeVehicule***, ***PrimeCommerciale***) et **des variables catégorielles** (SexeConducteur, FrequencePaiement, ClasseVehicule, etc.).


***PolicyId*** est un identifiant unique en format texte et ne sera pas utilisée pour l’entraînement.


### 2. Exploration et Nettoyage des donnés

Identifier et gérer les valeurs manquantes, les incohérences et les outliers.

In [ ]:
data.head(10) # Afficher les 10 premières lignes du dataset

In [ ]:
data.shape

In [ ]:
detect_missing_values(data)

**StatutMatrimonial** et **CodeProfession** ont un nombre important de valeurs manquantes, avec seulement 7372 valeurs renseignées sur 22481. 
La gestion des valeurs manquantes sera donc cruciale ici pour ne pas biaiser le modèle.

**NB**: il est préférable d'imputer les valeurs manquantes plutôt que de supprimer ces lignes, car cela entraînerait une perte d'environ 67% des données.

In [ ]:
print("les valeurs de StatutMatrimonial:",data['StatutMatrimonial'].unique())
print("les valeurs de CodeProfession :",data['CodeProfession'].unique())

**Options de gestion** :
* **Imputation par le mode** : Remplir les valeurs manquantes avec la valeur la plus courante, si l’une des catégories est très représentée.
* **Catégorie "Non spécifé"** : cette option permettrait de prendre en compte l'absence de cette information est significative, ce qui pourrait être pertinent.



**NB** : certains clients ne souhaitent pas déclarer leur statut

Donc Je vais remplir les valeurs manquantes par la valeur "Non spécifé"

In [ ]:
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Pré-traite les données en remplissant les valeurs manquantes pour certaines colonnes spécifiques.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données à pré-traiter.

    Returns:
        pd.DataFrame: Le DataFrame avec les valeurs manquantes remplies pour les colonnes 'StatutMatrimonial' et 'CodeProfession'.

    Étapes de pré-traitement :
        - Remplit les valeurs manquantes dans las colonne 'StatutMatrimonial'et 'CodeProfession' avec "Non spécifé".
    """
    
    df['StatutMatrimonial'].fillna("Non spécifié", inplace=True)
    df['CodeProfession'].fillna("Non spécifié", inplace=True)
    return df

In [ ]:
preprocess_data=preprocess_data(data) # Data with new vlaues
preprocess_data.head()

### 3. Analyse exploratoire des données (EDA)

Obtenir une vue d’ensemble des statistiques de base pour chaque variable afin de déceler des patterns ou des anomalies dans les données.

Identifier les variables les plus influentes sur PrimeCommerciale et de visualiser les relations entre les variables .

* **3.1 Analyse statistique descriptive**


In [ ]:
data.describe()


**AgeConducteur**

* **Moyenne**: 39.7 ans, avec un écart-type de 11.85 ans.
* **Distribution**  : L’âge des conducteurs varie de 18 à 87 ans. Les percentiles indiquent que 50 % des conducteurs ont moins de 37 ans, et 75 % ont moins de 47 ans.

* **Analyse** : La majorité des conducteurs sont dans un intervalle d'âge moyenne, ce qui est cohérent avec la démographie de nombreux assurés mais Les valeurs extrêmes, comme 87 ans, pourraient influencer la prime .

**BonusMalus**

* **Moyenne** : 63.24, avec un écart-type de 15.38.

* **Distribution** : La valeur minimale de BonusMalus est de 50, indiquant des conducteurs sans accidents récents. La valeur maximale de 156 peut correspondre à des conducteurs avec un historique de sinistres.

* **Analyse** : Cette variable a une signification directe sur le calcul de la prime. Un BonusMalus plus élevé peut être synonyme de primes plus coûteuses. La distribution montre que 75 % des assurés ont un BonusMalus inférieur à 72, ce qui suggère que la majorité des conducteurs ont un bon historique.



**AgeVehicule (Âge du véhicule)**

* **Moyenne** : 7.5 ans, avec un écart-type de 4.84 ans.
* **Distribution** : L’âge du véhicule varie de 0 (véhicules neufs) à 89 ans. Les quartiles indiquent que 50 % des véhicules ont moins de 7 ans, et 75 % moins de 10 ans.

* **Analyse** : La distribution est relativement concentrée autour de la moyenne, mais quelques véhicules très anciens peuvent nécessiter une attention particulière dans l'analyse, car ils pourraient influencer le modèle.



**PrimeCommerciale (Variable cible)**

* **Moyenne** : 420.79 , avec un écart-type de 219.26 
* **Distribution** : Les primes vont de 91  à 2902.3 . La médiane est de 375.1 €, ce qui signifie que la moitié des assurés paient moins de cette valeur.

* **Analyse** : La large gamme de primes reflète des variations importantes en fonction des caractéristiques des conducteurs et des véhicules. La queue de distribution des primes élevée (jusqu'à 2902 €) indique quelques valeurs extrêmes qui pourraient affecter la performance du modèle si elles sont mal gérées.



***Conclusion et Implications***

* **Normalisation** : Les variables AgeConducteur, BonusMalus, et AgeVehicule présentent des échelles différentes, il est donc pertinent de normaliser ou standardiser ces variables pour garantir une meilleure convergence des modèles linéaires.

* **Outliers** : Les valeurs extrêmes de AgeVehicule et PrimeCommerciale nécessiteront une attention particulière. 



* **2.2 Visualisation des distributions et relations**

In [ ]:
Numerical_colomuns=config['features']['numerical']
Categorical_columns=config['features']['categorical']

In [ ]:
plt.figure(figsize=(16, 8))
sns.color_palette("coolwarm")
for i, col in enumerate(Numerical_colomuns, 1):
          plt.subplot(2, 2, i)
          plt.grid(True)       
          plot_feature_distribution(preprocess_data,col)
plt.tight_layout()
plt.show()

**AgeConducteur** : La distribution de l’âge des conducteurs est légèrement asymétrique

**BonusMalus** : La distribution de BonusMalus est très asymétrique, 

**AgeVehicule** : La majorité des véhicules ont un âge inférieur à 10 ans . Les véhicules plus anciens sont rares, mais ils pourraient entraîner des primes plus élevées en raison d'un risque accru lié à l’usure.

**PrimeCommerciale** : La distribution de PrimeCommerciale  est fortement asymétrique, avec une majorité des primes situées entre 100 et 700 €. Il y a quelques valeurs élevées (au-delà de 2000 €), qui représentent des outliers potentiels. Cette distribution montre que la majorité des primes sont faibles, avec quelques valeurs extrêmes qui nécessiteront une attention particulière.

In [ ]:
plt.figure(figsize=(14,12))
sns.color_palette("dark")
target='PrimeCommerciale'
for i, col in enumerate(Numerical_colomuns[:-1],1):
          plt.subplot(3,1,i)
          plot_scatter(preprocess_data,col,target)
plt.tight_layout()
plt.show()

**PrimeCommerciale VS AgeConducteur** :

* La ligne de tendance montre que PrimeCommerciale diminue légèrement avec l'augmentation de l'âge du conducteur.


* AgeConducteur dans le modèle est pertinent pour capter la variation de la prime selon les catégories d'âge.

**PrimeCommerciale vs BonusMalus** :

* Le graphique montre une tendance positive, où une augmentation du BonusMalus entraîne une hausse de la prime. 

* Cette relation renforce le choix d’inclure BonusMalus comme variable prédictive clé, car elle influence directement les ajustements de prime.


**PrimeCommerciale vs AgeVehicule** :

* Le graphique de dispersion montre que les primes diminuent avec l'âge du véhicule, illustrant la relation inverse entre la valeur du véhicule et son âge.

* AgeVehicule doit être conservé dans le modèle, car il permet de prendre en compte le risque financier pour l'assureur en fonction de l'âge et de la valeur du véhicule.

In [ ]:
plt.figure(figsize=(10,6))
df_corr= preprocess_data[Numerical_colomuns]
plot_correlation_matrix(df_corr)


* La corrélation modérée de 0.38 entre BonusMalus et PrimeCommerciale montre que plus le BonusMalus est élevé, plus la prime d'assurance (PrimeCommerciale) augmente. 


* Une corrélation de -0.46 est observée entre AgeVehicule et PrimeCommerciale. Cette relation négative suggère que les véhicules plus anciens ont des primes d’assurance plus faibles

* La corrélation de -0.17 entre AgeConducteur et PrimeCommerciale est relativement faible, mais elle montre une légère tendance de réduction de la prime avec l'âge. Les conducteurs plus jeunes paient souvent des primes plus élevées, car ils sont considérés comme plus risqués.


**Conslusion**

* Les corrélations et les graphiques de dispersion démontre une compréhension claire des relations entre les variables et PrimeCommerciale. Les analyses montrent comment chaque variable influence la prime, ce qui est essentiel pour une modélisation efficace.

* En sélectionnant BonusMalus, AgeVehicule, et AgeConducteur, nous capturons les facteurs de risque liés au comportement du conducteur, à la valeur du véhicule et à l’expérience du conducteur. Ces choix sont en cohérence avec les pratiques d’assurance et les standards de l’industrie.

* Ces variables devraient contribuer de manière significative à la précision du modèle, car elles couvrent différents aspects du risque (comportement, valeur du véhicule, et profil d'âge). L’inclusion de ces variables est donc prévue pour optimiser la performance prédictive de PrimeCommerciale.

In [ ]:
plt.figure(figsize=(16, 20))
for i, column in enumerate(Categorical_columns, 1):
    plt.subplot(5, 2, i)
    sns.countplot(data=data, x=column, palette="viridis")
    plt.title(f"Distribution of {column}")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

plt.show()

**Encodage One-Hot** et **Traitement des Valeurs "Non spécifié"** : L'encodage one-hot des catégories dominantes permet de minimiser le biais dans le modèle, et le traitement des valeurs "Non spécifié" comme catégorie .

En intégrant des termes d’interaction entre des variables, on capture des relations complexes qui peuvent mieux expliquer la variabilité des primes.


In [ ]:
plt.figure(figsize=(15, 20))
# Plotting box plots for each continuous variable to visualize outliers
for i, col in enumerate(Categorical_columns, 1):
    plt.subplot(len(Categorical_columns)//2, 2, i)
    plot_box_plot(preprocess_data ,col,'PrimeCommerciale')

plt.tight_layout()
plt.show()

Les graphiques montrent que les primes d’assurance sont influencées par plusieurs facteurs. Les femmes, les veufs/divorcés, et les professions comme agriculteur ou commerçant paient généralement des primes plus élevées. Les véhicules plus puissants, chers, ou utilisés à des fins professionnelles, ainsi que ceux garés dans la rue ou situés dans certaines régions, augmentent également le coût. Les paiements mensuels sont les plus onéreux. Ces variables clés sont essentielles pour affiner la prédiction des primes.

### 4. Détection des Outliers


identifier les valeurs extrêmes qui peuvent fausser les résultats de la modélisation prédictive

In [ ]:
plt.figure(figsize=(16, 8))

for i, col in enumerate(Numerical_colomuns, 1):
    plt.subplot(2, 2, i)
    sns.boxplot(x=preprocess_data[col])
    plt.title(f'Box Plot of {col}')
    plt.grid(True)

plt.tight_layout()
plt.show()

En appliquant les méthodes de détection d’outliers sur les colonnes numériques, voici le nombre d'outliers détectés pour chaque variable 

In [ ]:
detect_outliers_summary(preprocess_data,Numerical_colomuns)

Les box-plots et les résultats de détection des outliers montrent un nombre significatif de valeurs aberrantes dans les différentes variables, en particulier pour la variable cible PrimeCommerciale. 



**Actions**  :

* **Capping des valeurs extrêmes** : Limiter les valeurs extrêmes à des seuils raisonnables pour PrimeCommerciale, BonusMalus, et AgeVehicule afin de réduire leur impact disproportionné sur le modèle.

* **Transformation Logarithmique** : Appliquer une transformation logarithmique à pour stabiliser l’effet des valeurs extrêmes tout en préservant les relations d'ordre.

In [ ]:
train_data=preprocess_data.copy()
columns_do_capping = config['features']['capping']
#scaled_columns = config['features']['scaled']
for col in columns_do_capping:
    train_data = cap_outliers(train_data, col) #Limite les valeurs extrêmes pour stabiliser les distributions.
train_data=log_transform(train_data,Numerical_colomuns) #Limite les valeurs extrêmes pour stabiliser les distributions
#train_data=scale_features(train_data,scaled_columns)
train_data.drop(columns=Numerical_colomuns, inplace=True) #Garde uniquement les versions transformées des colonnes.
print(train_data.info())

In [ ]:
train_data.describe()

In [ ]:
plt.figure(figsize=(16, 8))

# Plotting box plots for each continuous variable to visualize outliers
for i, col in enumerate(scaled_columns, 1):
    plt.subplot(2, 2, i)
    sns.boxplot(x=train_data[col])
    plt.title(f'Box Plot of {col}')

plt.tight_layout()
plt.show()

L'application du capping et de la transformation logarithmique a permis de :

Réduire l'impact des valeurs extrêmes dans chaque variable, en rendant les distributions plus symétriques et plus homogènes.
Améliorer la stabilité des variables pour la modélisation, en s’assurant que les valeurs aberrantes n’influencent pas de manière disproportionnée les prédictions.

## 5.Feature-engineering

* On commence par créer une interaction entre l’âge du conducteur et l’âge du véhicule, ce qui permet d’explorer l’effet combiné de ces deux variables.
* Ensuite, on simplifie l’analyse de l’âge du conducteur en le classant dans trois groupes : jeunes, matures et seniors.
* Une autre idée est d’ajouter une caractéristique qui mesure la différence entre le bonus-malus et l’âge du conducteur, pour détecter des tendances intéressantes.
* Pour exploiter les catégories d’âge, on les encode en valeurs numériques via un one-hot encoding.
* Enfin, on génère des variables polynomiales pour capturer des relations non linéaires entre certaines colonnes.

In [ ]:
def add_interaction_feature(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ajoute une caractéristique d'interaction entre l'âge du conducteur et l'âge du véhicule.
    """
    df['Driver_Vehicle_Age_Interaction'] = df['AgeConducteur_log'] * df['AgeVehicule_log']
    return df

def bin_age_conducteur(df: pd.DataFrame) -> pd.DataFrame:
    """
    Effectue un binning de 'AgeConducteur_log' en trois catégories : 'Young', 'Mature' et 'Senior'.
    """
    df['AgeConducteur_Binned'] = pd.cut(
        df['AgeConducteur_log'],
        bins=[-np.inf, np.log1p(25), np.log1p(50), np.inf],
        labels=['Young', 'Mature', 'Senior']
    )
    return df

def create_bonus_age_difference(df: pd.DataFrame) -> pd.DataFrame:
    """
    Crée une nouvelle caractéristique en soustrayant l'âge du conducteur du BonusMalus.
    """
    df['Bonus_Age_Difference'] = df['BonusMalus_log'] - df['AgeConducteur_log']
    return df

def encode_binned_age(df: pd.DataFrame) -> pd.DataFrame:
    """
    Encode la colonne 'AgeConducteur_Binned' en variables fictives (one-hot encoding).
    """
    encoder = OneHotEncoder(drop='first', sparse_output=False)
    encoded_columns = encoder.fit_transform(df[['AgeConducteur_Binned']])
    encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['AgeConducteur_Binned']))
    df = pd.concat([df, encoded_df], axis=1)
    df.drop(columns=['AgeConducteur_Binned'], inplace=True)
    return df

def add_polynomial_features(df: pd.DataFrame, columns: list, degree: int = 2) -> pd.DataFrame:
    """
    Génère des caractéristiques polynomiales pour les colonnes spécifiées.
    """
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    poly_features = poly.fit_transform(df[columns])
    poly_feature_names = poly.get_feature_names_out(columns)
    poly_df = pd.DataFrame(poly_features, columns=poly_feature_names)
    df = pd.concat([df, poly_df], axis=1)
    return df


In [ ]:
# Appliquer
train_data = add_interaction_feature(train_data)
train_data = bin_age_conducteur(train_data)
train_data = create_bonus_age_difference(train_data)
train_data = encode_binned_age(train_data)
#train_data = add_polynomial_features(train_data, ['AgeConducteur_log', 'BonusMalus_log']) # j'ai pas utilisé les ploy model

In [ ]:
train_data.to_csv(config['data']['train_cleaned_path'], index=False) # save data cleaned into a csv

In [ ]:
print("Feature-engineered Data:")
print(train_data.info())

 ## 6.Séparation des Données et Encodage Variables Catégorielle

Prépare les données pour la modélisation en appliquant l'encodage One-Hot avec OneHotEncoder aux variables catégorielles,

en supprimant les colonnes inutiles et en séparant les caractéristiques de la cible.

In [ ]:
X,y=preprocess_modeling(train_data, target_column='PrimeCommerciale_log', drop_columns=['PolicyId'])

## Étape 2 : Préparation, Entraînement, Evaluation et validation du modèle de régression

### 1.Séparation des Données d'Entraînement et de Validation

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### 2. Sélection de modèles et stratégie d'évaluation

* ### 2.1 Linear Models : 
   Modèles linéaires avec ou sans régularisation, adaptés pour des relations linéaires entre variables."

In [ ]:
models_ln=config['models']['linear_models']
cv = config['evaluation']['cv_folds']
scoring_metric=config['evaluation']['scoring_metric']
scoring=scoring_metric

models_ln

In [ ]:
model_classes = {
    'LinearRegression': LinearRegression,
    'RidgeRegression': Ridge,
    'LassoRegression': Lasso,
    'ElasticNetRegression': ElasticNet
}

# Define model parameters from your configuration

# Dictionary to store initialized models
models = {}

# Initialize models with parameters
for model_name, parameters in models_ln.items():
    model_class = model_classes.get(model_name)  # Retrieve model class
    if model_class:
        models[model_name] = model_class(**parameters)  # Initialize and store the model

# Display initialized models for verification
for name, model in models.items():
    print(f"{name}: {model}")

# Verify the models dictionary
models


In [ ]:

results = {
    'Model': [],
    'Mean MSE': [],
    'Mean R2': [],
    'Mean MAE':[]
}

for name, model in models.items():
    scores = evaluation_metrics(model, X_train, y_train, cv=cv)
    results['Model'].append(name)
    results['Mean MSE'].append(scores['Mean MSE'])
    results['Mean R2'].append(scores['Mean R2'])
    results['Mean MAE'].append(scores['Mean MAE'])
    


results_df = pd.DataFrame(results)
print("Évaluation des Modèles avec Validation Croisée")
print(results_df)





In [ ]:
train_sizes = np.linspace(0.1, 1.0, 10)
plt.figure(figsize=(12, 8))

for model_name, model in models.items():
    # Calcul des courbes d'apprentissage
    train_sizes, train_scores, test_scores = learning_curve(
        model, X, y, cv=5, train_sizes=train_sizes, scoring=scoring, n_jobs=-1
    )
    
    # Moyenne et écart-type des scores d'apprentissage et de validation
    train_scores_mean = -train_scores.mean(axis=1)
    train_scores_std = train_scores.std(axis=1)
    test_scores_mean = -test_scores.mean(axis=1)
    test_scores_std = test_scores.std(axis=1)

    # Tracer la courbe d'apprentissage pour chaque modèle
    plt.plot(train_sizes, train_scores_mean, 'o-', label=f'{model_name} - Entraînement')
    plt.plot(train_sizes, test_scores_mean, 'o--', label=f'{model_name} - Validation')

    # Afficher l'intervalle de confiance
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.2)
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.2)

# Personnalisation du graphique
plt.xlabel("Nombre d'exemples d'apprentissage")
plt.ylabel("Erreur Quadratique Moyenne (MSE)")
plt.title("Courbes d'Apprentissage des Modèles")
plt.legend(loc="best")
plt.grid()
plt.show()

In [ ]:
train_sizes, train_scores, val_scores = learning_curve(
  models['RidgeRegression']  , X, y, cv=cv, scoring='neg_mean_squared_error', train_sizes=np.linspace(0.1, 1.0, 10)
)

# Calcul de la moyenne et de l'écart-type des erreurs d'entraînement et de validation
train_errors_mean = -train_scores.mean(axis=1)
train_errors_std = train_scores.std(axis=1)
val_errors_mean = -val_scores.mean(axis=1)
val_errors_std = val_scores.std(axis=1)

# Tracé des courbes d'apprentissage
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_errors_mean, label="Erreur d'entraînement", color="blue")
plt.fill_between(train_sizes, train_errors_mean - train_errors_std, train_errors_mean + train_errors_std, alpha=0.1, color="blue")
plt.plot(train_sizes, val_errors_mean, label="Erreur de validation", color="red")
plt.fill_between(train_sizes, val_errors_mean - val_errors_std, val_errors_mean + val_errors_std, alpha=0.1, color="red")

# Ajout des labels et légendes
plt.xlabel("Taille de l'ensemble d'entraînement")
plt.ylabel("Erreur Quadratique Moyenne (MSE)")
plt.title("Courbes d'apprentissage avec validation croisée")
plt.legend(loc="best")
plt.show()

* ### 2.2 Méthodes d'ensemble

In [ ]:
X,y=preprocess_modeling(train_data, target_column='PrimeCommerciale_log', drop_columns=['PolicyId','AgeConducteur_log','BonusMalus_log'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.info()

In [ ]:
models_en=config["models"]["ensemble_methods"]
models_en

In [ ]:
train_data.head()

In [ ]:
models = {}

# Model classes mapping
model_classes = {
    "RandomForest": RandomForestRegressor,
    "GradientBoosting": GradientBoostingRegressor,
    "XGBoost": XGBRegressor,
}

# Initialize models using the configuration
for model_name, parameters in models_en.items():
    model_class = model_classes.get(model_name)  # Retrieve model class
    if model_class:
        try:
            models[model_name] = model_class(**parameters)  # Initialize and store the model
        except Exception as e:
            print(f"Error initializing {model_name}: {e}")

# Display initialized models for verification
for name, model in models.items():
    print(f"{name}: {model}")


In [ ]:
results = {
    'Model': [],
    'Mean MSE': [],
    'Mean R2': [],
    'Mean MAE':[]
}

for name, model in models.items():
    scores = evaluation_metrics(model, X_train, y_train, cv=cv)
    results['Model'].append(name)
    results['Mean MSE'].append(scores['Mean MSE'])
    results['Mean R2'].append(scores['Mean R2'])
    results['Mean MAE'].append(scores['Mean MAE'])
    


results_df = pd.DataFrame(results)
print("Évaluation des Modèles avec Validation Croisée")
print(results_df)



In [ ]:
train_sizes = np.linspace(0.1, 1.0, 10)
plt.figure(figsize=(12, 8))

for model_name, model in models.items():
    # Calcul des courbes d'apprentissage
    train_sizes, train_scores, test_scores = learning_curve(
        model, X, y, cv=5, train_sizes=train_sizes, scoring=scoring, n_jobs=-1
    )
    
    # Moyenne et écart-type des scores d'apprentissage et de validation
    train_scores_mean = -train_scores.mean(axis=1)
    train_scores_std = train_scores.std(axis=1)
    test_scores_mean = -test_scores.mean(axis=1)
    test_scores_std = test_scores.std(axis=1)

    # Tracer la courbe d'apprentissage pour chaque modèle
    plt.plot(train_sizes, train_scores_mean, 'o-', label=f'{model_name} - Entraînement')
    plt.plot(train_sizes, test_scores_mean, 'o--', label=f'{model_name} - Validation')

    # Afficher l'intervalle de confiance
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1)
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1)

# Personnalisation du graphique
plt.xlabel("Nombre d'exemples d'apprentissage")
plt.ylabel("Erreur Quadratique Moyenne (MSE)")
plt.title("Courbes d'Apprentissage des Modèles")
plt.legend(loc="best")
plt.grid()
plt.show()

In [ ]:
train_sizes, train_scores, val_scores = learning_curve(
models['RandomForest']  , X, y, cv=cv, scoring='neg_mean_squared_error', train_sizes=np.linspace(0.1, 1.0, 10)
)
# Calcul de la moyenne et de l'écart-type des erreurs d'entraînement et de validation
train_errors_mean = -train_scores.mean(axis=1)
train_errors_std = train_scores.std(axis=1)
val_errors_mean = -val_scores.mean(axis=1)
val_errors_std = val_scores.std(axis=1)

# Tracé des courbes d'apprentissage
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_errors_mean, label="Erreur d'entraînement", color="blue")
plt.fill_between(train_sizes, train_errors_mean - train_errors_std, train_errors_mean + train_errors_std, alpha=0.1, color="blue")
plt.plot(train_sizes, val_errors_mean, label="Erreur de validation", color="red")
plt.fill_between(train_sizes, val_errors_mean - val_errors_std, val_errors_mean + val_errors_std, alpha=0.1, color="red")

# Ajout des labels et légendes
plt.xlabel("Taille de l'ensemble d'entraînement")
plt.ylabel("Erreur Quadratique Moyenne (MSE)")
plt.title("Courbes d'apprentissage avec validation croisée")
plt.legend(loc="best")
plt.show()

In [ ]:
train_sizes, train_scores, val_scores = learning_curve(
models['XGBoost']  , X, y, cv=cv, scoring='neg_mean_squared_error', train_sizes=np.linspace(0.1, 1.0, 10)
)
# Calcul de la moyenne et de l'écart-type des erreurs d'entraînement et de validation
train_errors_mean = -train_scores.mean(axis=1)
train_errors_std = train_scores.std(axis=1)
val_errors_mean = -val_scores.mean(axis=1)
val_errors_std = val_scores.std(axis=1)

# Tracé des courbes d'apprentissage
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_errors_mean, label="Erreur d'entraînement", color="blue")
plt.fill_between(train_sizes, train_errors_mean - train_errors_std, train_errors_mean + train_errors_std, alpha=0.1, color="blue")
plt.plot(train_sizes, val_errors_mean, label="Erreur de validation", color="red")
plt.fill_between(train_sizes, val_errors_mean - val_errors_std, val_errors_mean + val_errors_std, alpha=0.1, color="red")

# Ajout des labels et légendes
plt.xlabel("Taille de l'ensemble d'entraînement")
plt.ylabel("Erreur Quadratique Moyenne (MSE)")
plt.title("Courbes d'apprentissage avec validation croisée")
plt.legend(loc="best")
plt.show()

* ### 2.3 NeuralNetwork MLP

In [ ]:
Mlp_model=config['models']['neural_networks']
Mlp_model['MLP']

In [ ]:
Mlp_model['MLP']=MLPRegressor(**Mlp_model['MLP'])

In [ ]:

results = {
    'Model': [],
    'Mean MSE': [],
    'Mean R2': [],
    'Mean MAE':[]
}

for name, model in Mlp_model.items():
    scores = evaluation_metrics(model, X_train, y_train, cv=cv)
    results['Model'].append(name)
    results['Mean MSE'].append(scores['Mean MSE'])
    results['Mean R2'].append(scores['Mean R2'])
    results['Mean MAE'].append(scores['Mean MAE'])
    


results_df = pd.DataFrame(results)
print("Évaluation des Modèles avec Validation Croisée")
print(results_df)

In [ ]:
train_sizes = np.linspace(0.1, 1.0, 10)
plt.figure(figsize=(12, 8))

    # Calcul des courbes d'apprentissage
train_sizes, train_scores, test_scores = learning_curve(
        model, X, y, cv=5, train_sizes=train_sizes, scoring=scoring, n_jobs=-1
    )
    
    # Moyenne et écart-type des scores d'apprentissage et de validation
train_scores_mean = -train_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
test_scores_mean = -test_scores.mean(axis=1)
test_scores_std = test_scores.std(axis=1)

    # Tracer la courbe d'apprentissage pour chaque modèle
plt.plot(train_sizes, train_scores_mean, 'o-', label= 'Entraînement')
plt.plot(train_sizes, test_scores_mean, 'o--', label=' Validation')

    # Afficher l'intervalle de confiance
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1)
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1)

# Personnalisation du graphique
plt.xlabel("Nombre d'exemples d'apprentissage")
plt.ylabel("Erreur Quadratique Moyenne (MSE)")
plt.title("Courbes d'Apprentissage des Modèles")
plt.legend(loc="best")
plt.grid(True)
plt.show()

In [ ]:
Model=Mlp_model['MLP'].fit(X_train,y_train)
y_pred=Model.predict(X_val)
print('Validation on set :',evaluate_on_validation_set(Model,y_val,y_pred))

Après une comparaison des modèles, le modèle Mlpregressor a démontré une performance supérieure grâce à un ajustement optimisé des hyperparamètres et à l'application de techniques efficaces pour limiter le surapprentissage (overfitting)."

### 3. Anlayse des Résidus

In [ ]:
import scipy.stats as stats
residuals = y_val - y_pred


# 1. Histogramme des résidus
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
sns.histplot(residuals, kde=True, color="blue")
plt.title("Histogramme des Résidus")
plt.xlabel("Résidu")
plt.ylabel("Fréquence")

# 2. Graphique des résidus vs valeurs prédites
plt.subplot(2, 2, 2)
plt.scatter(y_pred, residuals, alpha=0.5, color="green")
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Résidus vs Valeurs Prédites")
plt.xlabel("Valeurs Prédites")
plt.ylabel("Résidu")

# 3. QQ-plot pour la normalité des résidus
plt.subplot(2, 2, 3)
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("QQ-plot des Résidus")

# Afficher les graphiques
plt.tight_layout()
plt.show()


* Les résidus suivent une distribution normale, centrée autour de 0, indiquant que les erreurs sont bien réparties.

* Pour les Résidus vs Valeurs Prédictes : Pas de structure évidente, confirmant une bonne homogénéité des erreurs. Cependant, une légère dispersion est visible pour des valeurs prédites élevées.

* QQ-plot : Les résidus suivent globalement la ligne théorique, suggérant une normalité acceptable avec de légères déviations aux extrêmes.

En conclusion, le modèle est bien ajusté, avec des erreurs raisonnablement réparties et peu de biais visibles.

### 4. Sauvegarde du Modèle avec Chemin Configuré

In [ ]:
model_dir=config['data']['model_save_path']  
save_model_with_directory(Model,model_dir,'Model.pkl')

# Étape 3 :Exportation des prédictions :



In [ ]:
test_data.head()

In [ ]:
preprocess_data_test=preprocess_data(test_data) # Data with new vlaues
preprocess_data_test.head()

In [ ]:
# smae data preprocessing 
test_data_pre=preprocess_data_test.copy()
columns_do_capping = config['features']['capping'][:-1]
scaled_columns = config['features']['scaled'][:-1]
for col in columns_do_capping:
    test_data_pre = cap_outliers(test_data_pre, col) #Limite les valeurs extrêmes pour stabiliser les distributions.
test_data_pre=log_transform(test_data_pre,Numerical_colomuns[:-1]) #Limite les valeurs extrêmes pour stabiliser les distributions
#train_data=scale_features(train_data,scaled_columns)
test_data_pre.drop(columns=Numerical_colomuns[:-1], inplace=True) #Garde uniquement les versions transformées des colonnes.
print(test_data_pre.info())

In [ ]:
test_data_pre = add_interaction_feature(test_data_pre)
test_data_pre = bin_age_conducteur(test_data_pre)
test_data_pre = create_bonus_age_difference(test_data_pre)
test_data_pre = encode_binned_age(test_data_pre)
#test_data_pre = add_polynomial_features(test_data_pre, ['AgeConducteur_log', 'BonusMalus_log'])

In [ ]:
X_test=preprocess_modeling(test_data_pre,drop_columns=['PolicyId'])

In [ ]:
X_test['PuissanceVehicule_P6'] = 0

In [ ]:
expected_features = list(Model.feature_names_in_)

# Réorganiser les colonnes dans l’ordre attendu
X_test = X_test[expected_features]


In [ ]:
assert list(X_test.columns) == list(expected_features), "Les colonnes ne sont toujours pas alignées !"

In [ ]:
y_pred_log=Model.predict(X_test)

In [ ]:

y_pred=np.exp(y_pred_log)

In [ ]:
y_pred

In [ ]:
config['data']['output_path']

In [ ]:
Q1 = data['PrimeCommerciale'].quantile(0.25)
Q3 = data['PrimeCommerciale'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
bounds = {"lower": lower_bound, "upper": upper_bound}
y_pred = np.where(y_pred < bounds["lower"], bounds["lower"], y_pred)
y_pred = np.where(y_pred > bounds["upper"], bounds["upper"], y_pred) 

resultas=pd.DataFrame({
          "PolicyId":test_data['PolicyId'],
          "PrimeCommercialePred": y_pred
})


resultas.to_csv(config['data']['output_path'], index=False)

# Conclusion 

Résultats
- Le modèle le plus performant a été sélectionné après optimisation des hyperparamètres.
- Les métriques obtenues, comme le R² et l’erreur quadratique moyenne (RMSE), montrent une
 amélioration significative après nettoyage,l’ajout des nouvelles caractéristiques et l’optimisation.
 
Livrables
 - Un fichier CSV contenant les prédictions (PolicyId et PrimeCommercialePred) sur le jeu de données test.
 - Un notebook documentant l’ensemble des étapes, des analyses exploratoires à l’entraînement du modèle.
